In [1]:
import torch 
import numpy as np 
batch_size = 64
max_bag_length = 40
mean_bag_length = 20

In [2]:
random_bag_lengths = np.clip(
                    np.random.poisson(mean_bag_length, size=(batch_size)).astype(int),
                    1,
                    max_bag_length,
                )

In [3]:
attention_mask = torch.zeros((batch_size, max_bag_length), dtype=(torch.float32))
for i, l_ in enumerate(random_bag_lengths):
    attention_mask[i, :l_] = 1

In [22]:
attention_mask[:2,:]

tensor([[1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0.]])

In [4]:
input_tensor = torch.randn(
                (batch_size, max_bag_length, 384),
                dtype=(torch.float32),
            )

In [5]:
attention_mask.shape, input_tensor.shape

(torch.Size([64, 40]), torch.Size([64, 40, 384]))

In [6]:
attn = torch.randn(
                (batch_size, 12, max_bag_length, max_bag_length),
                dtype=(torch.float32),
            )
attn.shape

torch.Size([64, 12, 40, 40])

In [28]:
attn2 = attn + (-100000) * ( 1- attention_mask.reshape(batch_size, 1, 1, max_bag_length))
attn2 = attn2.softmax(dim=-1)
attn2.shape

torch.Size([64, 12, 40, 40])

In [30]:
attn2[0, 0, 2] 

tensor([0.0640, 0.0093, 0.0076, 0.0327, 0.0372, 0.0083, 0.0188, 0.0500, 0.3389,
        0.0064, 0.0242, 0.0529, 0.1783, 0.0153, 0.0697, 0.0093, 0.0302, 0.0159,
        0.0310, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000])

In [26]:
torch.sum(attention_mask[1] > 1e-4)

tensor(22)

In [38]:
torch.sum(attn2[1, 0, 0] > 1e-4)

tensor(22)

In [41]:
attn2[0, 0]

tensor([[0.0789, 0.1281, 0.0467,  ..., 0.0000, 0.0000, 0.0000],
        [0.0141, 0.4352, 0.0104,  ..., 0.0000, 0.0000, 0.0000],
        [0.0640, 0.0093, 0.0076,  ..., 0.0000, 0.0000, 0.0000],
        ...,
        [0.0658, 0.0385, 0.0551,  ..., 0.0000, 0.0000, 0.0000],
        [0.0116, 0.0050, 0.0490,  ..., 0.0000, 0.0000, 0.0000],
        [0.0748, 0.0337, 0.0078,  ..., 0.0000, 0.0000, 0.0000]])

In [27]:
seqlen = 10

In [12]:
mask = torch.full(
                (seqlen, seqlen), float("-inf"), 
            )

In [14]:
mask

tensor([[-inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf],
        [-inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf],
        [-inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf],
        [-inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf],
        [-inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf],
        [-inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf],
        [-inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf],
        [-inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf],
        [-inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf],
        [-inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf]])

In [15]:
mask = torch.triu(mask, diagonal=1)

In [16]:
mask

tensor([[0., -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf],
        [0., 0., -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf],
        [0., 0., 0., -inf, -inf, -inf, -inf, -inf, -inf, -inf],
        [0., 0., 0., 0., -inf, -inf, -inf, -inf, -inf, -inf],
        [0., 0., 0., 0., 0., -inf, -inf, -inf, -inf, -inf],
        [0., 0., 0., 0., 0., 0., -inf, -inf, -inf, -inf],
        [0., 0., 0., 0., 0., 0., 0., -inf, -inf, -inf],
        [0., 0., 0., 0., 0., 0., 0., 0., -inf, -inf],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., -inf],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])

In [17]:
mask = torch.hstack([
                torch.zeros((seqlen, 5),),
                mask
            ])

In [21]:
mask

tensor([[0., 0., 0., 0., 0., 0., -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf],
        [0., 0., 0., 0., 0., 0., 0., -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf],
        [0., 0., 0., 0., 0., 0., 0., 0., -inf, -inf, -inf, -inf, -inf, -inf, -inf],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., -inf, -inf, -inf, -inf, -inf, -inf],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., -inf, -inf, -inf, -inf, -inf],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., -inf, -inf, -inf, -inf],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., -inf, -inf, -inf],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., -inf, -inf],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., -inf],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])